In [1]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder,OrdinalEncoder
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [2]:
import pandas as pd
import numpy as np

In [6]:
df_train = pd.read_csv(r"C:\Users\luis_\Documents\GitHub\data-science-projects\house-price-prediction\train.csv")
df_test = pd.read_csv(r"C:\Users\luis_\Documents\GitHub\data-science-projects\house-price-prediction\test.csv")

## Data Wrangling

### Setting Correct Datatypes

#### 1) MSSubClass int > str
The "MSSubClass" category uses integers to specify types of dwelling, but these integers simply refer to categories.
We will change the datatype of this column from int to str.

<div style="float: left">

| Value  | Type of Dwelling |
|---|---|
| 20  | 1-STORY 1946 & NEWER ALL STYLES |
|  30 | 1-STORY 1945 & OLDER |
|  40 | 1-STORY W/FINISHED ATTIC ALL AGES |
etc.

</div>

In [7]:
# Transformation 1: int > str
df_train["MSSubClass"] = df_train["MSSubClass"].astype("str")

In [8]:
# Check
df_train["MSSubClass"]

0       60
1       20
2       60
3       70
4       60
        ..
1455    60
1456    20
1457    70
1458    20
1459    20
Name: MSSubClass, Length: 1460, dtype: object

#### 2) "Quality" columns str > int

As per the tables below, there are some columns with ordinal category values. However, this ordinality is not captured by the str datatype. We will convert these categories to ints using the mapping shown.

Columns: ExterQual\*, ExterCond\*, BsmtQual, BsmtCond, HeatingQC\*, KitchenQual\*, FireplaceQu, GarageQual, GarageCond, PoolQC <br />
\*Does not contain NA value

<div style="float: left">

| Category  | Ordinal Value  |
|---|---|
| Ex  | 5 |
| Gd  | 4 |
| TA  | 3 |
| Fa  | 2 |
| Po  | 1 |
| NA  | 0 |

Columns: BsmtFinType1, BsmtFinType2

| Category  | Ordinal Value  |
|---|---|
| GLQ  | 6 |
| ALQ  | 5 |
| BLQ  | 4 |
| Rec  | 3 |
| LwQ  | 2 |
| Unf  | 1 |
| NA  | 0 |


Columns: BsmtExposure

| Category  | Ordinal Value  |
|---|---|
| Gd  | 4 |
| Av  | 3 |
| Mn  | 2 |
| No  | 1 |
| NA  | 0 |
</div>

In [9]:
# Transformation 2: str > int
# Needs to have mapping from str values to int values

# Set 1
df_train[["ExterQual","ExterCond","BsmtQual","BsmtCond","HeatingQC","KitchenQual","FireplaceQu","GarageQual","GarageCond","PoolQC"]] = df_train[["ExterQual","ExterCond","BsmtQual","BsmtCond","HeatingQC","KitchenQual","FireplaceQu","GarageQual","GarageCond","PoolQC"]].replace(["NA","Po","Fa","TA","Gd","Ex"],[0,1,2,3,4,5])
# Set 2
df_train[["BsmtFinType1","BsmtFinType2"]] = df_train[["BsmtFinType1","BsmtFinType2"]].replace(["NA","Unf","LwQ","Rec","BLQ","ALQ","GLQ"],[0,1,2,3,4,5,6])
# Set 3
df_train["BsmtExposure"] = df_train["BsmtExposure"].replace(["Gd","Av","Mn","No","NA"],[4,3,2,1,0])

#### 3) "Year" columns to "YearsAgo"

This dataset contains columns pertaining to the years in which a property was built and modded. 
Keeping these years as is would not capture much of the relationship between the year column and the dependent variable. Consider the following example:

| Year  | Sale Price  |
|---|---|
| 1950  | 50,000 |
| 2000  | 200,000 |

To a human, we can intuit how a lot changed in these 50 years that would influence the sale price of a property. A model doesn't have this intuition, it just reads some incremental change in the variable. 2000 is just 50 more than 1950.

Note to self: I've changed my mind on this. The line of best fit would just have a y-intercept that's higher. We can test this.

In [ ]:
# Transform MoSold into "Fraction of Years Ago Sold"
df_train["MoSold"] = df_train["MoSold"].replace([int(x) for x in np.linspace(1,12,12)],np.linspace(0,1,12))

In [18]:
# Mix columns to include both Years and Months Ago Sold
df_train["YrMoSold"] = df_train["YrSold"] + df_train["MoSold"]

In [19]:
# Check
df_train["YrMoSold"]

0       2.909091
1       3.636364
2       2.272727
3       4.909091
4       2.000000
          ...   
1455    3.363636
1456    0.909091
1457    0.636364
1458    0.727273
1459    2.545455
Name: YrsMoAgoSold, Length: 1460, dtype: float64

In [20]:
# Drop YrSold, but keep MoSold to account for seasonality in sale prices
df_train = df_train.drop("YrSold",axis=1)

In [ ]:
# Changing columns from int > str
df_test["MSSubClass"]

In [ ]:
# preprocessing for numeric columns
imp_median = SimpleImputer(strategy="median", add_indicator=True)
scaler = StandardScaler()

In [ ]:
# preprocessing for categorical columns
imp_constant = SimpleImputer(strategy="constant")
ohe = OneHotEncoder(handle_unknown="ignore")

In [ ]:
# select columns by data type
num_cols = make_column_selector(dtype_include="number")
cat_cols = make_column_selector(dtype_exclude="number")

In [ ]:
# all preprocessing
preprocessor = make_column_transformer(
    (make_pipeline(imp_median, scaler), num_cols),
    (make_pipeline(imp_constant, ohe), cat_cols))

In [ ]:
# create a pipeline
pipe = make_pipeline(preprocessor, LogisticRegression())

In [ ]:
# cross-validate the pipeline
cross_val_score(pipe, X, y).mean()

In [ ]:
# fit pipeline and make predictions
pipe.fit(X,y)
pipe.predict(X,new)